In [1]:
import tkinter as tk
from tkinter import simpledialog
from tkinter import messagebox
import hashlib
import base64
import time
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding
import webbrowser

def generate_did():
    unique_id = hashlib.sha256(str(time.time()).encode()).hexdigest()
    did = "did:identity:" + unique_id
    return did

def generate_key_pair():
    private_key = ec.generate_private_key(ec.SECP256R1(), default_backend())
    public_key = private_key.public_key()
    private_key_pem = private_key.private_bytes(encoding=serialization.Encoding.PEM,
                                                format=serialization.PrivateFormat.TraditionalOpenSSL,
                                                encryption_algorithm=serialization.NoEncryption())
    public_key_pem = public_key.public_bytes(encoding=serialization.Encoding.PEM,
                                             format=serialization.PublicFormat.SubjectPublicKeyInfo)
    return private_key_pem, public_key_pem

def sign_data(private_key, data):
    private_key = serialization.load_pem_private_key(private_key, password=None, backend=default_backend())
    signature = private_key.sign(data, ec.ECDSA(hashes.SHA256()))
    return signature

def issue_anonymous_credential(subject_did, attributes, private_key):
    serialized_attributes = '|'.join([f"{key}:{value}" for key, value in attributes.items()]).encode()
    signature = sign_data(private_key, serialized_attributes)
    credential = {
        'subject_did': subject_did,
        'attributes': base64.b64encode(serialized_attributes).decode(),
        'signature': base64.b64encode(signature).decode()
    }
    return credential

def verify_age(credential):
    attributes = base64.b64decode(credential['attributes'])
    signature = base64.b64decode(credential['signature'])
    attributes = attributes.decode().split('|')
    attributes = {attr.split(':')[0]: attr.split(':')[1] for attr in attributes}
    if 'age' in attributes and int(attributes['age']) >= 18:
        return True
    else:
        return False

def select_profile(profile):
    if profile == "Adult":
        username = simpledialog.askstring("Credentials", "Enter your username:")
        password = simpledialog.askstring("Credentials", "Enter your password:", show='*')
        age = simpledialog.askinteger("Credentials", "Enter your age:")
        verify_adult_profile(username, password, age)
    elif profile == "Child":
        username = simpledialog.askstring("Credentials", "Enter your parent's username:")
        password = simpledialog.askstring("Credentials", "Enter your parent's password:", show='*')
        age = simpledialog.askinteger("Credentials", "Enter your age:")
        if  age:
            verify_child_profile(username, password, age)


def verify_adult_profile(username, password, age):
    if age >= 18:
        did = generate_did()
        private_key, public_key = generate_key_pair()
        attributes = {'name': username, 'age': str(age)}
        credential = issue_anonymous_credential(did, attributes, private_key)
        if verify_age(credential):
            messagebox.showinfo("Access Granted", "Welcome! You have access to adult content.")
            adult=1
            web_block(adult)
            webbrowser.open('https://www.google.com')
        else:
            messagebox.showerror("Access Denied", "Sorry, you don't meet the age requirement.")
    else:
        messagebox.showerror("Access Denied", "Invalid age.")

def verify_child_profile(username, password, age):
    adult=0
    web_block(adult)
    messagebox.showinfo("Access Granted", "Welcome! You have access to child-friendly content.")
    webbrowser.open('https://www.google.com')

def exit_program():
    if messagebox.askokcancel("Quit", "Do you want to quit?"):
        root.destroy()

def web_block(adult):
    hosts_path_windows = r"C:\Windows\System32\drivers\etc\hosts"
    hosts_path_unix = "/etc/hosts"
    # hosts_dir = "hosts"
    hosts_dir = hosts_path_windows
    redirect = "127.0.0.1"
    website_list = {
        "www.facebook.com"  
    }
    if not adult:
        with open(hosts_dir, 'r+') as file:
            content = file.read()
            for website in website_list:
                if website in content:
                    pass
                else: 
                    file.write(redirect + " " + website + "\n")
    else:
        with open(hosts_dir, 'r+') as file:
            content = file.readlines()
            file.seek(0)
            for line in content:
                if not any(website in line for website in website_list):
                    file.write(line)                
            file.truncate()

# Create the main window
root = tk.Tk()
root.title("Profile Selection")

# Create a frame for the profiles
profile_frame = tk.Frame(root)
profile_frame.pack(pady=20)

# Create profile buttons
adult_btn = tk.Button(profile_frame, text="Adult Profile", width=15, command=lambda: select_profile("Adult"))
adult_btn.grid(row=0, column=0, padx=10)

child_btn = tk.Button(profile_frame, text="Child Profile", width=15, command=lambda: select_profile("Child"))
child_btn.grid(row=0, column=1, padx=10)

# Create a button to exit the program
exit_button = tk.Button(root, text="Exit", command=exit_program)
exit_button.pack()

# Run the main event loop
root.mainloop()


In [1]:
import sqlite3
import hashlib

def anonimizar_credenciales(usuario, contraseña):
    # Concatenar el usuario y la contraseña
    credenciales = usuario + contraseña
    # Calcular el hash SHA-256
    hash_resultado = hashlib.sha256(credenciales.encode()).hexdigest()
    return hash_resultado

def crear_tabla():
    # Conectar a la base de datos (creará un archivo si no existe)
    conn = sqlite3.connect('credenciales.db')
    c = conn.cursor()

    # Crear la tabla de credenciales
    c.execute('''CREATE TABLE IF NOT EXISTS credenciales (
                 id INTEGER PRIMARY KEY,
                 usuario TEXT NOT NULL,
                 contraseña TEXT NOT NULL,
                 edad INTEGER NOT NULL)''')

    # Guardar los cambios y cerrar la conexión
    conn.commit()
    conn.close()

def insertar_credencial(usuario, contraseña, edad):
    # Conectar a la base de datos
    conn = sqlite3.connect('credenciales.db')
    c = conn.cursor()

    # Anonimizar las credenciales
    credenciales_anonimizadas = anonimizar_credenciales(usuario, contraseña)

    # Insertar las credenciales anonimizadas y la edad en la tabla
    c.execute("INSERT INTO credenciales (usuario, contraseña, edad) VALUES (?, ?, ?)",
              (usuario, credenciales_anonimizadas, edad))

    # Guardar los cambios y cerrar la conexión
    conn.commit()
    conn.close()

def verificar_credenciales(usuario, contraseña):
    # Conectar a la base de datos
    conn = sqlite3.connect('credenciales.db')
    c = conn.cursor()

    # Anonimizar las credenciales para compararlas con la base de datos
    credenciales_anonimizadas = anonimizar_credenciales(usuario, contraseña)

    # Buscar las credenciales en la base de datos
    c.execute("SELECT edad FROM credenciales WHERE usuario=? AND contraseña=?", (usuario, credenciales_anonimizadas))
    resultado = c.fetchone()

    # Cerrar la conexión
    conn.close()

    if resultado:
        edad = resultado[0]
        if edad > 18:
            return True
        else:
            return False
    else:
        return False

# Crear la tabla si no existe
crear_tabla()

# Insertar algunas credenciales de ejemplo
insertar_credencial("usuario1", "contraseña1", 20)
insertar_credencial("usuario2", "contraseña2", 16)

# Verificar si las credenciales tienen una edad mayor a 18 años
print(verificar_credenciales("usuario1", "contraseña1"))  # Debería devolver True
print(verificar_credenciales("usuario2", "contraseña2"))  # Debería devolver False


True
False


In [ ]:
import tkinter as tk
from tkinter import messagebox
import sqlite3
import hashlib
import webbrowser

def anonimizar_credenciales(usuario, contraseña):
    # Concatenar el usuario y la contraseña
    credenciales = usuario + contraseña
    # Calcular el hash SHA-256
    hash_resultado = hashlib.sha256(credenciales.encode()).hexdigest()
    return hash_resultado

def crear_tabla():
    # Conectar a la base de datos (creará un archivo si no existe)
    conn = sqlite3.connect('credenciales.db')
    c = conn.cursor()

    # Crear la tabla de credenciales
    c.execute('''CREATE TABLE IF NOT EXISTS credenciales (
                 id INTEGER PRIMARY KEY,
                 usuario TEXT NOT NULL,
                 contraseña TEXT NOT NULL,
                 edad INTEGER NOT NULL)''')

    # Guardar los cambios y cerrar la conexión
    conn.commit()
    conn.close()

def crear_usuario():
    usuario = entry_usuario.get()
    contraseña = entry_contraseña.get()
    edad = entry_edad.get()

    # Verificar si se ingresaron tanto el usuario, la contraseña y la edad
    if usuario and contraseña and edad:
        # Anonimizar las credenciales
        credenciales_anonimizadas = anonimizar_credenciales(usuario, contraseña)

        # Insertar el usuario en la base de datos
        conn = sqlite3.connect('credenciales.db')
        c = conn.cursor()
        c.execute("INSERT INTO credenciales (usuario, contraseña, edad) VALUES (?, ?, ?)",
                  (usuario, credenciales_anonimizadas, edad))
        conn.commit()
        conn.close()

        messagebox.showinfo("Éxito", "Usuario creado exitosamente.")
    else:
        messagebox.showerror("Error", "Por favor ingrese tanto el usuario, la contraseña como la edad.")

def ingresar_pagina_web():
    usuario = entry_usuario.get()
    contraseña = entry_contraseña.get()

    if usuario and contraseña:
        conn = sqlite3.connect('credenciales.db')
        c = conn.cursor()
        c.execute("SELECT edad FROM credenciales WHERE usuario=? AND contraseña=?",
                  (usuario, anonimizar_credenciales(usuario, contraseña)))
        resultado = c.fetchone()
        conn.close()

        if resultado:
            edad = resultado[0]
            if edad >= 18:
                webbrowser.open("https://www.google.com")  # Cambia esto por la URL de la página a la que deseas redirigir
            else:
                messagebox.showinfo("Acceso denegado", "Lo sentimos, debes ser mayor de 18 años para acceder.")
        else:
            messagebox.showerror("Error", "Credenciales incorrectas.")
    else:
        messagebox.showerror("Error", "Por favor ingrese tanto el usuario como la contraseña.")

# Crear la tabla si no existe
crear_tabla()

# Crear la ventana principal
root = tk.Tk()
root.title("Verificación de Edad")

# Crear los widgets
label_usuario = tk.Label(root, text="Usuario:")
label_contraseña = tk.Label(root, text="Contraseña:")
label_edad = tk.Label(root, text="Edad:")
entry_usuario = tk.Entry(root)
entry_contraseña = tk.Entry(root, show="*")  # La contraseña se muestra como asteriscos
entry_edad = tk.Entry(root)
boton_crear_usuario = tk.Button(root, text="Crear Usuario", command=crear_usuario)
boton_ingresar_pagina = tk.Button(root, text="Ingresar a la Página", command=ingresar_pagina_web)

# Ubicar los widgets en la ventana
label_usuario.grid(row=0, column=0, sticky="e")
label_contraseña.grid(row=1, column=0, sticky="e")
label_edad.grid(row=2, column=0, sticky="e")
entry_usuario.grid(row=0, column=1)
entry_contraseña.grid(row=1, column=1)
entry_edad.grid(row=2, column=1)
boton_crear_usuario.grid(row=3, column=0, pady=5)
boton_ingresar_pagina.grid(row=3, column=1, pady=5)

# Iniciar el bucle principal de la aplicación
root.mainloop()


In [ ]:
import tkinter as tk
from tkinter import ttk
import sqlite3
import hashlib

# Función para encriptar una cadena usando SHA-256
def encriptar_cadena(cadena):
    return hashlib.sha256(cadena.encode()).hexdigest()

# Función para desencriptar los datos
def desencriptar_datos(usuario, contraseña):
    # Aquí implementarías el método de desencriptación adecuado
    # Para este ejemplo, simplemente devolveremos los datos originales
    return usuario, contraseña

def ver_usuarios():
    # Conectar a la base de datos
    conn = sqlite3.connect('credenciales.db')
    c = conn.cursor()

    # Obtener todos los usuarios de la base de datos
    c.execute("SELECT * FROM credenciales")
    usuarios = c.fetchall()

    # Cerrar la conexión
    conn.close()

    # Crear ventana para mostrar usuarios
    ventana_usuarios = tk.Toplevel()
    ventana_usuarios.title("Usuarios Registrados")

    # Crear tabla para mostrar usuarios
    tabla = ttk.Treeview(ventana_usuarios, columns=("Usuario", "Contraseña", "Edad"))
    tabla.heading("#0", text="ID")
    tabla.heading("Usuario", text="Usuario")
    tabla.heading("Contraseña", text="Contraseña")
    tabla.heading("Edad", text="Edad")

    # Insertar usuarios en la tabla
    for usuario in usuarios:
        # Desencriptar la contraseña antes de mostrarla
        usuario_encriptado, contraseña_encriptada = desencriptar_datos(usuario[1], usuario[2])
        tabla.insert("", "end", text=usuario[0], values=(usuario_encriptado, contraseña_encriptada, usuario[3]))

    tabla.pack(expand=True, fill="both")

# Crear la ventana principal
root = tk.Tk()
root.title("Verificación de Usuarios")

# Crear los widgets
boton_ver_usuarios = tk.Button(root, text="Ver Usuarios", command=ver_usuarios)

# Ubicar los widgets en la ventana
boton_ver_usuarios.pack(pady=10)

# Iniciar el bucle principal de la aplicación
root.mainloop()
